# Assignment

In [3]:
# Import 

import numpy as np
import matplotlib.pyplot as plt
from degree_freedom_queen import *
from degree_freedom_king1 import *
from degree_freedom_king2 import *
from generate_game import *
from Chess_env import *


size_board = 4

## The Environment

You can find the environment in the file Chess_env, which contains the class Chess_env. To define an object, you need to provide the board size considered as input. In our example, size_board=4. 
Chess_env is composed by the following methods:

1. Initialise_game. The method initialises an episode by placing the three pieces considered (Agent's king and queen, enemy's king) in the chess board. The outputs of the method are described below in order.

     S $\;$ A matrix representing the board locations filled with 4 numbers: 0, no piece in that position; 1, location of the 
     agent's king; 2 location of the queen; 3 location of the enemy king.
     
     X $\;$ The features, that is the input to the neural network. See the assignment for more information regarding the            definition of the features adopted. To personalise this, go into the Features method of the class Chess_env() and change        accordingly.
     
     allowed_a $\;$ The allowed actions that the agent can make. The agent is moving a king, with a total number of 8                possible actions, and a queen, with a total number of $(board_{size}-1)\times 8$ actions. The total number of possible actions correspond      to the sum of the two, but not all actions are allowed in a given position (movements to locations outside the borders or      against chess rules). Thus, the variable allowed_a is a vector that is one (zero) for an action that the agent can (can't)      make. Be careful, apply the policy considered on the actions that are allowed only.
     

2. OneStep. The method performs a one step update of the system. Given as input the action selected by the agent, it updates the chess board by performing that action and the response of the enemy king (which is a random allowed action in the settings considered). The first three outputs are the same as for the Initialise_game method, but the variables are computed for the position reached after the update of the system. The fourth and fifth outputs are:

     R $\;$ The reward. To change this, look at the OneStep method of the class where the rewards are set.
     
     Done $\;$ A variable that is 1 if the episode has ended (checkmate or draw).
     
     
3. Features. Given the chessboard position, the method computes the features.

This information and a quick analysis of the class should be all you need to get going. The other functions that the class exploits are uncommented and constitute an example on how not to write a python code. You can take a look at them if you want, but it is not necessary.






In [4]:
## INITIALISE THE ENVIRONMENT

env=Chess_Env(size_board)

In [33]:
## PRINT 5 STEPS OF AN EPISODE CONSIDERING A RANDOM AGENT

S,X,allowed_a=env.Initialise_game()                       # INTIALISE GAME

print(S)                                                  # PRINT CHESS BOARD (SEE THE DESCRIPTION ABOVE)

print('check? ',env.check)                                # PRINT VARIABLE THAT TELLS IF ENEMY KING IS IN CHECK (1) OR NOT (0)
print('dofk2 ',np.sum(env.dfk2_constrain).astype(int))    # PRINT THE NUMBER OF LOCATIONS THAT THE ENEMY KING CAN MOVE TO


for i in range(5):
    
    a,_=np.where(allowed_a==1)                  # FIND WHAT THE ALLOWED ACTIONS ARE
    a_agent=np.random.permutation(a)[0]         # MAKE A RANDOM ACTION

    S,X,allowed_a,R,Done=env.OneStep(a_agent)   # UPDATE THE ENVIRONMENT

    ## PRINT CHESS BOARD AND VARIABLES
    print('')
    print(S)
    print(R,'', Done)
    print('check? ',env.check)
    print('dofk2 ',np.sum(env.dfk2_constrain).astype(int))
    
    
    # TERMINATE THE EPISODE IF Done=True (DRAW OR CHECKMATE)
    if Done:
        break
        


[[3 0 0 0]
 [0 0 1 0]
 [0 0 0 0]
 [0 2 0 0]]
check?  0
dofk2  1
idk 32

[[0 0 0 0]
 [3 0 0 0]
 [0 0 0 1]
 [0 2 0 0]]
0  0
check?  0
dofk2  1
idk 32

[[3 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 2 1 0]]
0  0
check?  0
dofk2  1
idk 32

[[3 0 0 0]
 [0 2 0 0]
 [0 0 0 0]
 [0 0 1 0]]
1  1
check?  1
dofk2  0


In [53]:
# PERFORM N_episodes=1000 EPISODES MAKING RANDOM ACTIONS AND COMPUTE THE AVERAGE REWARD AND NUMBER OF MOVES 

S,X,allowed_a=env.Initialise_game()
N_episodes=1000

# VARIABLES WHERE TO SAVE THE FINAL REWARD IN AN EPISODE AND THE NUMBER OF MOVES 
R_save_random = np.zeros([N_episodes, 1])
N_moves_save_random = np.zeros([N_episodes, 1])

for n in range(N_episodes):
    
    S,X,allowed_a=env.Initialise_game()     # INITIALISE GAME
    Done=0                                  # SET Done=0 AT THE BEGINNING
    i=1                                     # COUNTER FOR THE NUMBER OF ACTIONS (MOVES) IN AN EPISODE
    
    # UNTIL THE EPISODE IS NOT OVER...(Done=0)
    while Done==0:
        
        # SAME AS THE CELL BEFORE, BUT SAVING THE RESULTS WHEN THE EPISODE TERMINATES 
        
        a_base,_=np.where(allowed_a==1)
        print(a_base)
        a_agent_base=np.random.permutation(a_base)[0]

        S,X,allowed_a,R,Done=env.OneStep(a_agent_base)
                
        if Done:
            
            R_save_random[n]=np.copy(R)
            N_moves_save_random[n]=np.copy(i)

            break

        i=i+1                               # UPDATE THE COUNTER



# AS YOU SEE, THE PERFORMANCE OF A RANDOM AGENT ARE NOT GREAT, SINCE THE MAJORITY OF THE POSITIONS END WITH A DRAW 
# (THE ENEMY KING IS NOT IN CHECK AND CAN'T MOVE)

print('Random_Agent, Average reward:',np.mean(R_save_random),'Number of steps: ',np.mean(N_moves_save_random))



Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
[ 0  1  2  9 15 24 26 27 28 29]
[ 0  1  2 15 24 27 28]
[ 0  1  2  9 15 24 26 27 28 29]
[ 0  1  3  6  9 10 12 18 26 27]
[ 6  9 10 12 18 21 26 27 28 29]
[ 6  7  9 18 24 25 27 29 31]
[ 3  4  5  6  7  9 18 19 21 24 25 26 28 29 30]
[ 3  9 10 18 21 25 31]
[ 6  9 18 21 27]
[ 3  4  5  6  7 18 25 27 31]
[ 6  7  8 19 20 24 25 29]
[ 3  6 18 20 25 26 27 30 31]
[ 0  3  4  6  7  8 12 26 27]
[ 0  1  3 12 25 26 27 30 31]
[ 0  1  3  6 12 13 18 25 26 27 30 31]
[ 0  1  2  6 12 13 14 25 26 27 31]
[ 0  1  6  7  8 12 25 26 30]
[ 3  4  6 12 18 19 26 30]
[ 3  9 15 21 25 26 27 29 30 31]
[ 0  1  3  9 10 26 27]
[ 3  6  7  9 12 13 21 24 26 27 28]
[ 0  3  4  9 10 15 24 25 26 27 30 31]
[ 0  3  4  6  9 10 12 18 25 26 27 30 31]
[ 3  4  6  9 12 15 18 19 21 27 31]
[ 0  3  6 12 15 16 18 25 26 27 30 31]
[ 3  9 15 16 21 24 27 29 31]
[ 0  3  4 12 15 18 24 25 29 31]
[ 3  4  5  6 24 25 27 31]
[ 3  9 10 18 25]
[ 0  3  4  6 15 18 25 27]
[ 1  2  6  9 10 12 25]
[

In [44]:
# INITIALISE THE PARAMETERS OF YOUR NEURAL NETWORK AND...
# PLEASE CONSIDER TO USE A MASK OF ONE FOR THE ACTION MADE AND ZERO OTHERWISE IF YOU ARE NOT USING VANILLA GRADIENT DESCENT...
# WE SUGGEST A NETWORK WITH ONE HIDDEN LAYER WITH SIZE 200. 


S,X,allowed_a=env.Initialise_game()
N_a=np.shape(allowed_a)[0]   # TOTAL NUMBER OF POSSIBLE ACTIONS

N_in=np.shape(X)[0]    ## INPUT SIZE
N_h=200                ## NUMBER OF HIDDEN NODES


## INITALISE YOUR NEURAL NETWORK...


# HYPERPARAMETERS SUGGESTED (FOR A GRID SIZE OF 4)

epsilon_0 = 0.2     # STARTING VALUE OF EPSILON FOR THE EPSILON-GREEDY POLICY
beta = 0.00005      # THE PARAMETER SETS HOW QUICKLY THE VALUE OF EPSILON IS DECAYING (SEE epsilon_f BELOW)
gamma = 0.85        # THE DISCOUNT FACTOR
eta = 0.0035        # THE LEARNING RATE

N_episodes = 100000 # THE NUMBER OF GAMES TO BE PLAYED 

# SAVING VARIABLES
R_save = np.zeros([N_episodes, 1])
N_moves_save = np.zeros([N_episodes, 1])


Qtable = np.random.randn(1, N_a)/1000

In [26]:
print(Qtable.shape)


(1, 32)


In [62]:
def EpsilonGreedy_Policy(Qvalues, epsilon, allowed_a):
    allowed_i,_=np.where(allowed_a==1)

    N_a=np.shape(Qvalues)[0]

    rand_value=np.random.uniform(0,1)

    rand_a=rand_value<epsilon

    if rand_a==True:
        a = np.random.choice(allowed_i)
        #a=np.random.randint(0,N_a)
    else:

        a=np.argmax(Qvalues)
            
    return a

In [55]:
print(Qtable)
print(X)


[[-1.30142386e-03  1.62948219e-03 -6.11936391e-04 -1.15952603e-03
   9.17354793e-04 -4.20312403e-04 -1.25481195e-04 -1.69112152e-04
  -6.03152720e-04  1.03151184e-03 -1.69210351e-03 -2.31371175e-04
  -1.57616905e-03 -3.20361337e-04  1.89912724e-03 -8.73771078e-04
  -2.02694994e-03  1.96344203e-03  6.41087000e-04 -3.82281462e-04
  -2.07201873e-05 -2.55449975e-04  1.99569402e-03 -4.71541797e-04
  -3.82648441e-04 -4.66369681e-04 -1.20142213e-03 -4.16907733e-04
   1.73840987e-03  1.63148236e-03  6.54810607e-05  1.79602592e-03]]
[]


In [66]:
# TRAINING LOOP BONE STRUCTURE...
# I WROTE FOR YOU A RANDOM AGENT (THE RANDOM AGENT WILL BE SLOWER TO GIVE CHECKMATE THAN AN OPTIMISED ONE, 
# SO DON'T GET CONCERNED BY THE TIME IT TAKES), CHANGE WITH YOURS ...

eligibility_trace = True
if eligibility_trace:
    lamb=0.3
    eta=0.08


for n in range(N_episodes):

    epsilon_f = epsilon_0 / (1 + beta * n)   ## DECAYING EPSILON
    Done=0                                   ## SET DONE TO ZERO (BEGINNING OF THE EPISODE)
    i = 1                                    ## COUNTER FOR NUMBER OF ACTIONS
    
    S,X,allowed_a=env.Initialise_game()      ## INITIALISE GAME
    #print(n)                                 ## REMOVE THIS OF COURSE, WE USED THIS TO CHECK THAT IT WAS RUNNING
    
    X_all = np.array([X])

    ind_x = -1
    count = -1
    for el in X_all:
        count += 1
        if np.array_equal(el, X):
            ind_x = count
              
    
    if ind_x == -1:
        np.append(X_all, [X], axis=0)
        Q = np.random.randn(1, N_a)/1000
        np.append(Qtable, Q, axis=0)

    

    nallowed_i,_=np.where(allowed_a==0)
    Qvalues = np.copy(Qtable[ind_x, :])
    for i in nallowed_i:
        Qvalues[i] = -100
    print(Qvalues)
    a = EpsilonGreedy_Policy(Qvalues, epsilon_0, allowed_a) #make an action

    while Done==0:                           ## START THE EPISODE
        
        
        ## THIS IS A RANDOM AGENT, CHANGE IT...
       

        S_next,X_next,allowed_a_next,R,Done=env.OneStep(a)

        R_save[n] = np.copy(R)
        N_moves_save[n] = np.copy(i)
        
        ## THE EPISODE HAS ENDED, UPDATE...BE CAREFUL, THIS IS THE LAST STEP OF THE EPISODE
        if Done==1:
            
            
            break
        
        
        
        
        # IF THE EPISODE IS NOT OVER...
        else:
            
            ## ONLY TO PUT SUMETHING
            PIPPO=1
        
        ind_x1 = -1
        count = -1
        for el in X_all:
            count += 1
            if np.array_equal(el, X_next):
                ind_x1 = count             
      
        if ind_x1 == -1:
            np.append(X_all, [X_next], axis=0)
            Q = np.random.randn(1, N_a)/1000
            np.append(Qtable, Q, axis=0)

        nallowed_i,_=np.where(allowed_a_next==0)
        Qvalues = np.copy(Qtable[ind_x1, :])
        for i in nallowed_i:
            Qvalues[i] = -100
        print(Qvalues)
        a1 = EpsilonGreedy_Policy(Qvalues, epsilon_0, allowed_a_next) #make an action

        # Compute the delta
        delta=R+gamma*Qtable[ind_x1, a1]-Qtable[ind_x, a]
        
        # Update the Qvalues
        Qtable[ind_x, a]=Qtable[ind_x, a]+eta*delta

        # NEXT STATE AND CO. BECOME ACTUAL STATE...     
        S=np.copy(S_next)
        X=np.copy(X_next)
        allowed_a=np.copy(allowed_a_next)
        
        i += 1  # UPDATE COUNTER FOR NUMBER OF ACTIONS

        
        
    
    

[-1.30142386e-03 -1.00000000e+02 -1.00000000e+02 -1.15952603e-03
 -1.00000000e+02 -1.00000000e+02 -1.00000000e+02 -1.00000000e+02
 -1.00000000e+02  1.06801254e-03 -1.00000000e+02 -1.00000000e+02
 -1.57616905e-03 -3.20361337e-04 -1.00000000e+02 -8.73771078e-04
 -1.00000000e+02 -1.00000000e+02 -1.00000000e+02 -1.00000000e+02
 -1.00000000e+02 -2.55449975e-04 -1.00000000e+02 -1.00000000e+02
 -1.00000000e+02 -4.66369681e-04 -1.20142213e-03 -4.16907733e-04
 -1.00000000e+02 -1.00000000e+02  6.54810607e-05  1.79602592e-03]
[-1.00000000e+02 -1.00000000e+02 -1.00000000e+02 -1.15952603e-03
 -1.00000000e+02 -1.00000000e+02  1.28281861e-04 -1.00000000e+02
 -1.00000000e+02 -1.00000000e+02 -1.00000000e+02 -1.00000000e+02
 -1.00000000e+02 -1.00000000e+02 -1.00000000e+02 -1.00000000e+02
 -1.00000000e+02 -1.00000000e+02  6.41087000e-04 -1.00000000e+02
 -1.00000000e+02 -2.55449975e-04 -1.00000000e+02 -1.00000000e+02
 -3.82648441e-04 -4.66369681e-04 -1.20142213e-03 -1.00000000e+02
  1.73840987e-03 -1.0000

IndexError: ignored